# Setup

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Manipulating dataframes
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# System Options
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Date and Time
import datetime
from datetime import timedelta

# Uploading user data from JSON files
import json
from pandas.io.json import json_normalize

# Cluster
from sklearn.cluster import KMeans

# Working directory
path = '/content/drive/My Drive/Data Wizards/Data/'

# Read Files

In [0]:
user_json = []
for line in open(path + '28Aug-UserAll.json', 'r'):
    user_json.append(json.loads(line))

#peghasilan = income
#pekerjaan = work
#pendidikan = education

# Convert to pandas
user = json_normalize(user_json[0])
#user.to_csv(path + '28Aug-UserAll.csv', index = False)
user.head()

In [0]:
shopper_json = []
for line in open(path + '28Aug-Shopper.json', 'r'):
    shopper_json.append(json.loads(line))

# Convert to pandas
shopper = json_normalize(shopper_json[0])
shopper.head()

In [0]:
email_raw = pd.read_excel(path + 'EmailLogs.xlsx')
email_raw.head()

In [0]:
TS_raw = pd.read_excel(path + 'trusrtingSocialData.xlsx')
TS_raw.head()

In [0]:
def find_fate(df):
  for i in df.columns:
    print("'"+i+"':","'KEEP', ")

# Cleaning User Data


In [0]:
# Read date and time
def dateparse(x):
  try:
    return pd.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ')
  except:
    try:
      return pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
    except:
      return np.nan

# For missing values in CardType
def lowercase(x):
  try:
    return x.lower()
  except:
    return None

# Define data cleaning function
def Clean(df, FATE):
  DROP = []
  OHE = pd.DataFrame()
  coords_count = 0

  if list(df.columns) != list(FATE.keys()):
    raise Exception("Cleaning and fate columns aren't equal")

  for col in df.columns:
    if FATE[col] == 'DROP':
      DROP.append(col)
      
    elif FATE[col] == 'DATE':
      df[col] = df[col].apply(dateparse)
      temp_array = np.array([(i - datetime.datetime(1970, 1, 1)) / datetime.timedelta(days=1) for i in df[col]])
      df[col] = temp_array.astype('float')  

    elif FATE[col] == 'phone':
      df[col] = df[col].apply(lambda x: str(x)[1:])
      df.rename(columns = {col:'Phone ID'}, inplace = True)
      FATE[col] = 'ID'

    elif (FATE[col] == 'OHE'):
      ohe = pd.get_dummies(df[col], prefix = col)
      DROP.append(col)
      OHE = pd.concat([OHE, ohe], axis=1)
  
  # Edit the dataframe
  df = pd.concat([df, OHE], axis=1)
  df.drop(DROP, axis = 1, inplace = True)
  return df

In [0]:
FATE = {
    		'_id': 'ID',
				'activated': 'DROP',
				'card': 'KEEP',
				'createdAt': 'DATE',
				'credit': 'DROP',
				'danaVerifiedAccount': 'OHE',
				'defaultPayment': 'OHE',
				'detail.birthdate': 'DATE',
				'detail.birthplace': 'KEEP', #Drop for now
				'detail.email': 'ID',
				'detail.gender': 'OHE',
				'detail.industri': 'OHE',
				'detail.name': 'DROP',
				'detail.peghasilan': 'OHE',
				'detail.pekerjaan': 'OHE',
				'detail.pendidikan': 'OHE',
				'detail.penghasilan': 'OHE',
				'emailVerified': 'OHE',
				'emergencyContact.mobileNumber': 'DROP',
				'emergencyContact.name': 'DROP',
				'emergencyContact.type': 'OHE',
				'ktp.image': 'DROP', # Drop for now
				'ktp.number': 'DROP', # Mostly empty
				'ktp.status': 'OHE', # What's this?
				'loc.coordinates': 'KEEP',
				'loc.type': 'DROP',
				'mobileNumber': 'phone',
				'npwp': 'DROP', # Mostly empty
				'registrationLoc.coordinates': 'KEEP',
				'registrationLoc.type': 'DROP',
				'remainingCredit': 'DROP',
				'selfie': 'DROP', # Drop for now
				'status': 'DROP', # Pending, etc. 
				'updatedAt': 'DROP',
				'verify.applicant.notes': 'DROP',
				'verify.applicant.passed': 'DROP',
				'verify.emergencyContact.notes': 'DROP',
				'verify.emergencyContact.passed': 'DROP',
}


In [0]:
user_raw = json_normalize(user_json[0])
user = Clean(user_raw, FATE)
user.head()

## Location

In [39]:
# Unpack the coordinates from the list
def parse_coords(series):
  Long = []
  Lat = []
  for row in series:
    if row in [[], np.nan]:
      Long.append(0)
      Lat.append(0)
      continue
    
    Long.append(float(row[0]))
    Lat.append(float(row[1]))
  return pd.DataFrame({'Long':Long,'Lat':Lat})

# Clean the location at last use, and at registration
user[['Long','Lat']] = parse_coords(user['loc.coordinates'])
user[['Long_reg','Lat_reg']] = parse_coords(user['registrationLoc.coordinates'])

user.drop(['loc.coordinates','registrationLoc.coordinates'], inplace = True, axis = 1)
user['Long'].head()

0     98.767909
1    106.794929
2    107.603402
3    106.896405
4    107.880047
Name: Long, dtype: float64

## Unpack credit card details

In [0]:
# Turn each card into a row in a dataframe
cards = pd.concat([pd.DataFrame(x) for x in user['card']], keys = user['_id']).reset_index(level=1, drop=True)

# OHE categorical values
cards['type'] = cards['type'].apply(lambda x: x.lower())
cards = pd.concat([cards, pd.get_dummies(cards[['bank','type']]), ], axis = 1)

# Groupby cardholder and summarise information
cards = cards.groupby(['_id']).agg({'bank_VISA':'sum', 'bank_bni':'sum','type_credit':'sum','type_debit':'sum'})
cards.rename(columns={'bank_VISA':'VISA_cards', 'bank_bni':'BNI_cards','type_credit':'SumCredit','type_debit':'SumDebit'})
cards.head()

In [0]:
# Merge user and cards
user = pd.merge(user,cards, how = 'outer', on = '_id')
user.drop(['card','danaVerifiedAccount_False'], axis = 1, inplace = True)

# Fill missing bank
user.loc[:,['bank_VISA','bank_bni','type_credit','type_debit']] = user.loc[:,['bank_VISA','bank_bni','type_credit','type_debit']].fillna(0,axis = 1)

user.head(10)

# Cleaning Shopper

In [0]:
# Clean
FATE = {'_id': 'KEEP',
        'createdAt': 'DATE',
				'status': 'DROP',
				'store._id': 'DROP', # what's this? Is there an index table?
				'store.merchant._id': 'DROP',
				'store.merchant.name': 'KEEP', # Do not OHE yet
				'store.name': 'DROP',
				'termins': 'KEEP', # Merge with shopper on...?
				'total': 'KEEP',
				'transactionNumber': 'DROP',
				'updatedAt': 'DROP',
				'user._id': 'ID', # Merge with user on _id
				'user.detail.email': 'DROP',
				'user.detail.name': 'DROP',
				'user.mobileNumber': 'DROP',
				'user.selfie': 'DROP',
				'user.status': 'DROP'}
        
shopper = json_normalize(shopper_json[0])
shopper = Clean(shopper, FATE)
shopper.head()

In [0]:
merchants = pd.DataFrame(shopper['store.merchant.name'].value_counts())
merchants.to_excel(path + 'merchants.xls')

## Unpack Termins

In [0]:
FATE = {'_id': 'KEEP', # Merge with shopper on this
        'lateFee': 'KEEP', 
        'Installment_Number': 'KEEP', 
        'Installment_Amount': 'KEEP', 
        'Due_Date': 'DATE', 
        'lastNotified': 'DROP', 
        'notified': 'DROP', 
        'lastChecked': 'DROP', # Drop - the whole thing is missing
        'lastFineCharged': 'DROP', 
        'Payment_Date': 'DATE', # Fix this bad boy 
        'method': 'OHE', 
        'payment_id': 'DROP',
        'status_code': 'DROP', 
        'status': 'KEEP', # Money received true/false
        'order_id': 'DROP', 
        'paymentGateway': 'DROP'}

# Unpack termins
termins = pd.concat([pd.DataFrame(x) for x in shopper['termins']], keys = shopper['_id']).reset_index(level = 1, drop = True)
termins.drop('_id', axis = 1, inplace = True)
termins.reset_index(inplace=True)

# Unpack due
due = termins.due.apply(pd.Series)
termins = pd.concat([termins,due], axis = 1)
termins =  termins.rename(columns = {'date': 'Due_Date', 'number':'Installment_Number','total':'Installment_Amount'})

# Unpack paid
paid = termins['paid'].apply(pd.Series)
termins = pd.concat([termins, paid], axis = 1)
termins.rename(columns = {'date': 'Payment_Date'}, inplace = True)

# Drop dictionary columns inside termins
termins.drop(['due','paid'], axis = 1, inplace = True)

# Clean
termins = Clean(termins, FATE)

# Target variable
termins['Lateness'] = termins.Due_Date - termins.Payment_Date

termins.head()

## Summarise EmpatKali History

In [0]:
# Merge all EmpatKali loan history
history = shopper.merge(termins, how='outer',on="_id")

# Get the days since the last loan
temp_array = np.array([(pd.to_datetime('today') - datetime.datetime(1970, 1, 1)) / datetime.timedelta(seconds=1)])
temp_array = temp_array/(60*60*24)
history['Last_Loan'] = temp_array - history['createdAt']

# Encode late fees paid
history['paidLateFee'] = np.where(history.lateFee > 0, 1, 0)

# OHE Merchant
ohe = pd.get_dummies(history['store.merchant.name'], prefix = 'Merchant')

# Drop columns you don't need anymore
drop = ['termins','_id','store.merchant.name']
history = pd.concat([history, ohe], axis=1).drop(drop, axis = 1)

# Rename
Names = {'user._id': '_id','createdAt':'Loan_Date','total':'Avg_Loan','status':'Paid'}
history.rename(columns=Names, inplace = True)

# Drop rows where lateness is missing
history.dropna(inplace=True)
history.reset_index(inplace=True, drop = True)

history.head(10)

In [0]:
Agg =  {'_id': 'count',
        'Avg_Loan': 'mean', 
        'lateFee': 'sum',
        'paidLateFee':'sum',
        'Installment_Amount': 'max', 
        'method_MASTERCARD CREDIT': 'sum', 
        'method_VISA CREDIT': 'sum', 
        'method_credit_card': 'sum', 
        'method_dana': 'sum', 
        'method_vabni': 'sum', 
        'Lateness': 'mean', 
        'Last_Loan': 'min', 
        'Merchant_Alvilabelonline': 'sum', 
        'Merchant_Brodo': 'sum', 
        'Merchant_DekadeTime': 'sum', 
        'Merchant_Groot Pay Indonesia': 'sum', 
        'Merchant_JW ETNIK': 'sum', 
        'Merchant_KHAKIKAKIKU': 'sum', 
        'Merchant_Karatvan': 'sum', 
        'Merchant_Kasual': 'sum', 
        'Merchant_OTIV': 'sum', 
        'Merchant_Pale Buddy': 'sum', 
        'Merchant_Phillip Works': 'sum', 
        'Merchant_Sagara Boot Maker': 'sum', 
        'Merchant_Saint Barkley Shoes': 'sum', 
        'Merchant_Seymour': 'sum', 
        'Merchant_Wellbourn': 'sum', 
        'Merchant_bambooblonde': 'sum', 
        'Merchant_dapoza': 'sum', 
        'Merchant_footstep Footwear': 'sum', 
        'Merchant_gudangimport': 'sum', 
        'Merchant_guteninc': 'sum', 
        'Merchant_kidzclusive': 'sum', 
        'Merchant_tulisan': 'sum', 
        'Merchant_vininc': 'sum'}

history2 = history.groupby(['_id']).agg(Agg).rename(columns = {'_id':'Count_Loans'})
history2.reset_index(inplace = True)
history2.head()

In [0]:
# Get a dataframe of all repayments merged with their transactions
user2 = user.merge(history, how = 'inner', on="_id")
user2.head()

#141 columns

# Cleaning Email Data

In [0]:
email_raw = pd.read_excel(path + 'EmailLogs.xlsx')
email_raw.head()

In [0]:
# OHE
tipe_ohe = pd.get_dummies(email_raw.tipe, prefix='email')
email = pd.concat([email_raw.email, tipe_ohe], axis =1)

# Groupby each user
email = email.groupby(['email']).sum()
email.reset_index(inplace = True)
email = email.rename(columns = {'email': 'detail.email'} )
email.head()

In [0]:
# Merge with user
user3 = user2.merge(email, how = 'left', on = 'detail.email')
user3.fillna(0,axis = 1, inplace=True)
user3.head()

# Trusting Social

In [0]:
# One hot encode credit score
TS = TS_raw.loc[:,['SCORE','MOBILE_NUMBER','SCORE_CATEGORY']]
dummies = pd.get_dummies(TS['SCORE_CATEGORY'], prefix = "TrustSoc_", prefix_sep = '')
TS = pd.concat([TS, dummies], axis = 1)
TS.drop('SCORE_CATEGORY', axis = 1, inplace = True)

# Prepare MOBILE_NUMBER for merge with user data
TS['Phone ID'] = TS['MOBILE_NUMBER'].apply(str)
TS.drop('MOBILE_NUMBER',axis =1, inplace = True)
user4 = user3.merge(TS, on = 'Phone ID', how = 'left')

TS.head()

In [52]:
# Drop the ID columns from training data
drop = ['detail.birthdate', 'detail.birthplace','detail.email', 'Phone ID']
user4.drop(drop, axis = 1, inplace = True)

# Impute 0 or average for customers without trusting social data
user4.loc[:,['TrustSoc_A','TrustSoc_B','TrustSoc_C','TrustSoc_D']] = user4.loc[:,['TrustSoc_A','TrustSoc_B','TrustSoc_C','TrustSoc_D']].fillna(0)
user4.loc[:,['SCORE']] = user4.loc[:,['SCORE']].fillna(user4.SCORE.mean())

user4.columns

Index(['_id', 'createdAt', 'danaVerifiedAccount_True', 'defaultPayment_card',
       'defaultPayment_dana', 'detail.gender_f', 'detail.gender_m',
       'detail.industri_industri1', 'detail.industri_industri10',
       'detail.industri_industri11',
       ...
       'Merchant_vininc', 'email_bounce', 'email_delivered', 'email_open',
       'email_reject', 'SCORE', 'TrustSoc_A', 'TrustSoc_B', 'TrustSoc_C',
       'TrustSoc_D'],
      dtype='object', length=143)

In [0]:
today = datetime.date.today().strftime('%Y%m%d')
user4.to_csv(path + 'Train_' + today + '.xlsx', index=False)

# K Means Clustering

In [54]:
print(user4.shape)
user4.head()

(406, 143)


,_id,createdAt,danaVerifiedAccount_True,defaultPayment_card,defaultPayment_dana,detail.gender_f,detail.gender_m,detail.industri_industri1,detail.industri_industri10,detail.industri_industri11,detail.industri_industri12,detail.industri_industri13,detail.industri_industri14,detail.industri_industri15,detail.industri_industri16,detail.industri_industri17,detail.industri_industri18,detail.industri_industri19,detail.industri_industri2,detail.industri_industri20,detail.industri_industri21,detail.industri_industri22,detail.industri_industri23,detail.industri_industri24,detail.industri_industri3,detail.industri_industri4,detail.industri_industri5,detail.industri_industri6,detail.industri_industri7,detail.industri_industri8,detail.industri_industri9,detail.peghasilan_gol5,detail.pekerjaan_POLRI,detail.pekerjaan_TNI,detail.pekerjaan_administrative,detail.pekerjaan_art-creative,detail.pekerjaan_bumd,detail.pekerjaan_bumn,detail.pekerjaan_business-analyst,detail.pekerjaan_business-development,...,method_MASTERCARD CREDIT,method_VISA CREDIT,method_credit_card,method_dana,method_vabni,Lateness,Last_Loan,paidLateFee,Merchant_Alvilabelonline,Merchant_Brodo,Merchant_DekadeTime,Merchant_Groot Pay Indonesia,Merchant_JW ETNIK,Merchant_KHAKIKAKIKU,Merchant_Karatvan,Merchant_Kasual,Merchant_OTIV,Merchant_Pale Buddy,Merchant_Phillip Works,Merchant_Sagara Boot Maker,Merchant_Saint Barkley Shoes,Merchant_Seymour,Merchant_Wellbourn,Merchant_bambooblonde,Merchant_dapoza,Merchant_footstep Footwear,Merchant_gudangimport,Merchant_guteninc,Merchant_kidzclusive,Merchant_tulisan,Merchant_vininc,email_bounce,email_delivered,email_open,email_reject,SCORE,TrustSoc_A,TrustSoc_B,TrustSoc_C,TrustSoc_D
0,5d6607ef9d73e0152361c31a,18136.201287,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,-0.291655,11.668103,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,8.0,14.0,0.0,850.00000,1.0,0.0,0.0,0.0
1,5d63085ba18f3843d9f9990e,18133.926989,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0.107717,12.493598,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,7.0,8.0,0.0,719.59887,0.0,0.0,0.0,0.0
2,5d63085ba18f3843d9f9990e,18133.926989,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,-0.291655,12.493598,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,7.0,8.0,0.0,719.59887,0.0,0.0,0.0,0.0
3,5d613318a18f3843d9f9969b,18132.536575,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,-0.291655,11.439094,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,7.0,21.0,0.0,719.59887,0.0,0.0,0.0,0.0
4,5d5b9025216d69050da06e8f,18128.261172,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,-0.291655,19.742857,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,6.0,5.0,0.0,718.00000,1.0,0.0,0.0,0.0


In [55]:
# Fit KMeans 
X_train = user4.drop('_id', axis = 1)
KMns_Args = {'n_clusters': 3, 'random_state': 1}
kmeans = KMeans(**KMns_Args) 
kmeans.fit(X_train)

for clust in range(3):
  count = np.count_nonzero(kmeans.labels_ == clust)
  print('Class',str(clust),'- Count',str(count))

Class 0 - Count 257
Class 1 - Count 10
Class 2 - Count 139


In [56]:
# Group data by cluster
user5 = user4.assign(Cluster = kmeans.labels_)
user5 = user5.groupby(['Cluster']).mean()
user5.head()

,createdAt,danaVerifiedAccount_True,defaultPayment_card,defaultPayment_dana,detail.gender_f,detail.gender_m,detail.industri_industri1,detail.industri_industri10,detail.industri_industri11,detail.industri_industri12,detail.industri_industri13,detail.industri_industri14,detail.industri_industri15,detail.industri_industri16,detail.industri_industri17,detail.industri_industri18,detail.industri_industri19,detail.industri_industri2,detail.industri_industri20,detail.industri_industri21,detail.industri_industri22,detail.industri_industri23,detail.industri_industri24,detail.industri_industri3,detail.industri_industri4,detail.industri_industri5,detail.industri_industri6,detail.industri_industri7,detail.industri_industri8,detail.industri_industri9,detail.peghasilan_gol5,detail.pekerjaan_POLRI,detail.pekerjaan_TNI,detail.pekerjaan_administrative,detail.pekerjaan_art-creative,detail.pekerjaan_bumd,detail.pekerjaan_bumn,detail.pekerjaan_business-analyst,detail.pekerjaan_business-development,detail.pekerjaan_copy-writer,...,method_MASTERCARD CREDIT,method_VISA CREDIT,method_credit_card,method_dana,method_vabni,Lateness,Last_Loan,paidLateFee,Merchant_Alvilabelonline,Merchant_Brodo,Merchant_DekadeTime,Merchant_Groot Pay Indonesia,Merchant_JW ETNIK,Merchant_KHAKIKAKIKU,Merchant_Karatvan,Merchant_Kasual,Merchant_OTIV,Merchant_Pale Buddy,Merchant_Phillip Works,Merchant_Sagara Boot Maker,Merchant_Saint Barkley Shoes,Merchant_Seymour,Merchant_Wellbourn,Merchant_bambooblonde,Merchant_dapoza,Merchant_footstep Footwear,Merchant_gudangimport,Merchant_guteninc,Merchant_kidzclusive,Merchant_tulisan,Merchant_vininc,email_bounce,email_delivered,email_open,email_reject,SCORE,TrustSoc_A,TrustSoc_B,TrustSoc_C,TrustSoc_D
Cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,18064.588109,0.984436,0.824903,0.143969,0.015564,0.898833,0.015564,0.0,0.000000,0.015564,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.003891,0.0,0.0,0.000000,0.653696,0.003891,0.019455,0.0,0.0,0.007782,0.108949,0.171206,0.000000,0.015564,0.0,0.0,0.0,0.0,0.07393,0.015564,0.0,0.011673,0.0,...,0.003891,0.147860,0.136187,0.0,0.712062,3.757379,38.258596,0.124514,0.054475,0.050584,0.003891,0.0,0.015564,0.023346,0.015564,0.007782,0.023346,0.023346,0.027237,0.031128,0.0,0.070039,0.031128,0.015564,0.062257,0.011673,0.451362,0.023346,0.038911,0.015564,0.003891,0.0,504.922179,349.556420,1.595331,721.875047,0.603113,0.003891,0.003891,0.0
1,18061.055040,1.000000,0.300000,0.700000,0.400000,0.600000,0.000000,0.0,0.000000,0.000000,0.300000,0.0,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.600000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.300000,0.0,0.700000,1.361855,38.287393,0.400000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,72.000000,50.100000,0.000000,678.619209,0.000000,0.000000,0.300000,0.0
2,18059.611359,0.575540,0.201439,0.388489,0.100719,0.899281,0.028777,0.0,0.093525,0.028777,0.043165,0.0,0.028777,0.079137,0.0,0.014388,0.014388,0.035971,0.0,0.0,0.014388,0.093525,0.035971,0.028777,0.0,0.0,0.086331,0.093525,0.237410,0.043165,0.028777,0.0,0.0,0.0,0.0,0.00000,0.007194,0.0,0.000000,0.0,...,0.000000,0.014388,0.237410,0.0,0.748201,1.115200,53.475851,0.165468,0.000000,0.705036,0.021583,0.0,0.014388,0.115108,0.000000,0.000000,0.043165,0.000000,0.086331,0.000000,0.0,0.000000,0.007194,0.000000,0.000000,0.000000,0.000000,0.007194,0.000000,0.000000,0.000000,0.0,57.035971,52.568345,0.460432,718.338577,0.079137,0.035971,0.007194,0.0
